<center>
<h1> Seach in arabic documents </h1>

</centre>

## Import tools

In [1]:
!pip install PyArabic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install opendatasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import numpy as np
import pandas as pd 
import tensorflow as tf
import nltk
from pyarabic.araby import tokenize, is_arabicrange, strip_tashkeel
from nltk.corpus import stopwords
import re
import string
from string import digits
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from scipy import spatial
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import os
import opendatasets as od
import pickle

## Download the dataset
[Click here to learn how to download dataset via API](https://www.geeksforgeeks.org/how-to-import-kaggle-datasets-directly-into-google-colab/#:~:text=Method%202%3A%20By%20easily%20downloading,folder%20to%20Google%20Colab%20Notebook.)

In [4]:
od.download("https://www.kaggle.com/datasets/haithemhermessi/sanad-dataset")

Skipping, found downloaded files in "./sanad-dataset" (use force=True to force download)


## Load the data from directory

In [5]:
classes = os.listdir('/content/sanad-dataset') 

In [6]:
classes

['Finance', 'Sports', 'Medical', 'Tech', 'Culture', 'Politics', 'Religion']

In [7]:
def load_data():
  return tf.keras.utils.text_dataset_from_directory(
    '/content/sanad-dataset',
    labels='inferred',
    label_mode='int',
    class_names=None,
    max_length=None,
    shuffle=True)

In [8]:
dataset = load_data()

Found 45500 files belonging to 7 classes.


## Turn the dataset into DataFrame

In [9]:
x=[]
y=[]
for text_batch, label_batch in dataset:
    for i in range(len(text_batch)):
        s=text_batch.numpy()[i].decode("utf-8") 
        x.append(s)
        y.append(dataset.class_names[label_batch.numpy()[i]])
print(len(x))
print(len(y))

45500
45500


In [10]:
df = pd.DataFrame({'Article': x, 'class':y})

In [11]:
len(df)

45500

In [12]:
df.head()

,Article,class
0,سينطلق سائق مرسيدس أي إم جي الألماني نيكو روزب...,Sports
1,إعداد:محمد إبراهيم لا شك أن تاريخ الإمارات عام...,Religion
2,تقدم أكسيس كوميونكيشنز سلسلتين جديدين من المنت...,Tech
3,"دبي ""الخليج"":تزامناً مع احتفالية العالم بيوم ا...",Medical
4,لا شك أن محبي نادي أي سي ميلان الإيطالي يتطلعو...,Sports


## Check for nulls and secret killers

In [13]:
df.isnull().sum() / len(df) # Print percentage of NULL

Article    0.0
class      0.0
dtype: float64

In [14]:
df.duplicated().sum() / len(df)

0.00032967032967032967

### Removing the duplicates

In [15]:
df.drop_duplicates(inplace=True)

## Preprocessing and cleaning text

 Most of cleaning regex and functions are belongs to this [gist](https://github.com/motazsaad/process-arabic-text/blob/master/clean_arabic_text.py) on github

In [16]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [17]:
arabic_nums =  ['٠', '١', '٢', '٣', '٤', '٥', '٦', '٧', '٨', '٩']

In [18]:
ara_stop_words = list(set(stopwords.words('arabic'))) + arabic_nums

In [19]:
len(ara_stop_words)

711

In [20]:
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,،'{}~¦+«»`|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

In [21]:
punctuations_list = list(set(punctuations_list)) # Add all together

In [22]:
# This function for deleting any encoded emojis in the text
def remove_emoji(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

In [23]:
# This function should delete any non arabic text and numbers, also will remove 'tashkeel'
def clean_arabic_txt(text):
  res = remove_emoji(text)
  res = ' '.join([word for word in text if word not in punctuations_list])
  res = tokenize(text, conditions=is_arabicrange, morphs=strip_tashkeel)
  res = ' '.join(res)
  res = ''.join([words for words in res if words not in arabic_nums])
  return res

<h2> Test the clean function </h2>

In [24]:
TEXT = 'السلام عليكم 1234 ورحمة الله،،!@# ٧٧٧'

In [25]:
clean_arabic_txt(TEXT)

'السلام عليكم ورحمة الله '

In [26]:
# Clean the text and add it in a new pandas column, this probably will take sometime
df['articles_cln'] = df['Article'].apply(clean_arabic_txt)

In [27]:
df.head()

,Article,class,articles_cln
0,سينطلق سائق مرسيدس أي إم جي الألماني نيكو روزب...,Sports,سينطلق سائق مرسيدس أي إم جي الألماني نيكو روزب...
1,إعداد:محمد إبراهيم لا شك أن تاريخ الإمارات عام...,Religion,إعداد محمد إبراهيم لا شك أن تاريخ الإمارات عام...
2,تقدم أكسيس كوميونكيشنز سلسلتين جديدين من المنت...,Tech,تقدم أكسيس كوميونكيشنز سلسلتين جديدين من المنت...
3,"دبي ""الخليج"":تزامناً مع احتفالية العالم بيوم ا...",Medical,دبي الخليج تزامنا مع احتفالية العالم بيوم المت...
4,لا شك أن محبي نادي أي سي ميلان الإيطالي يتطلعو...,Sports,لا شك أن محبي نادي أي سي ميلان الإيطالي يتطلعو...


## Load the model and the encoder

In [28]:
svm_model = pickle.load(open('/content/svm_model.sav', 'rb'))

In [29]:
label_encoder = pickle.load(open('/content/encoder.sav', 'rb'))

In [30]:
df['class'] = label_encoder.transform(df['class'])

In [31]:
df.head()

,Article,class,articles_cln
0,سينطلق سائق مرسيدس أي إم جي الألماني نيكو روزب...,5,سينطلق سائق مرسيدس أي إم جي الألماني نيكو روزب...
1,إعداد:محمد إبراهيم لا شك أن تاريخ الإمارات عام...,4,إعداد محمد إبراهيم لا شك أن تاريخ الإمارات عام...
2,تقدم أكسيس كوميونكيشنز سلسلتين جديدين من المنت...,6,تقدم أكسيس كوميونكيشنز سلسلتين جديدين من المنت...
3,"دبي ""الخليج"":تزامناً مع احتفالية العالم بيوم ا...",2,دبي الخليج تزامنا مع احتفالية العالم بيوم المت...
4,لا شك أن محبي نادي أي سي ميلان الإيطالي يتطلعو...,5,لا شك أن محبي نادي أي سي ميلان الإيطالي يتطلعو...


## Build the searcher


In [32]:
# Shuffle the dataFrame to make sure we will sample many words as possible
from sklearn.utils import shuffle
df = shuffle(df)

In [33]:
vectorizer = TfidfVectorizer()

In [34]:
def cos_sim(vec1, vec2):
  sim = 1 - spatial.distance.cosine(list(vec1), list(vec2))
  return sim

In [93]:
def searcher(sentence):
  articles = [] # Store the article
  scores = []  # Store the score of the article to sort it later
  class_pred = svm_model.predict([sentence])[0] # Predect which class the user is search in
  # Filter the dataFrame depend on the predicted class only
  filter = (df['class'] == class_pred)
  df_filtrd = df[filter]

  # Build the tf-idf features
  docs_vec = vectorizer.fit_transform(df_filtrd['articles_cln'])
  sentence_vec = vectorizer.transform([sentence]).toarray()

  # Compare each vector with the sentence vector, and retrive the highest scores
  for i, vec in enumerate(docs_vec):
    score = cos_sim(vec.toarray(), sentence_vec)
    scores.append(round(score, 2))
    articles.append(df_filtrd.iloc[i]['articles_cln'])
  
  results_df = pd.DataFrame({'Articles': articles, 'Score': scores})
  return results_df


## Test the searcher

In [107]:
user_sentence = input("Enter the query: ")

Enter the query: الثورة السورية


In [108]:
search_df = searcher(user_sentence)

In [109]:
search_df.head()

,Articles,Score
0,أعلن الأمين العام لمنظمة التعاون الإسلامي أكمل...,0.04
1,بحث ولي العهد السعودي ووزير الدفاع الأمير سلما...,0.00
2,نفت مصادر أمنية مطلعة أمس ، أن تكون السلطات ال...,0.00
3,تمكن الأمن المغربي ، أمس الثلاثاء ، من توقيف ع...,0.04
4,نفى حزب النور ، الذراع السياسية للدعوة السلفية...,0.00


In [110]:
sorted_df = search_df.sort_values(by=['Score'], ascending=False)

In [111]:
sorted_df.head()

,Articles,Score
2667,بدأت قوى المعارضة السورية أمس ، اجتماعاتها في ...,0.21
6236,أصدر المرشد الأعلى للجمهورية الإسلامية في إيرا...,0.20
4977,شكل معارضون سوريون مجلسا وطنيا لمواجهة نظام دم...,0.18
4777,اعتقلت السلطات السورية ، أمس ، اثنين من أعضاء ...,0.18
2213,أقدم شاب يمني مساء أول أمس ، على إشعال النار ف...,0.18


## Display the articles

In [112]:
from IPython.display import Markdown, display
for i in range(5):
    display(Markdown('> '+sorted_df['Articles'][i]))
    print('--------------------------------------')

> أعلن الأمين العام لمنظمة التعاون الإسلامي أكمل الدين إحسان أوغلي ، رفض منظمته التدخل العسكري الأجنبي في الأزمة السورية ، وحذر من دفع الأمور نحو حرب أهلية وقال على هامش مشاركته في أعمال مؤتمر أصدقاء سوريا في تونس ، إن ما يقوم به النظام السوري من أعمال عنف هو أمر مرفوض ، لكن لا يعني ذلك تشريع التدخل العسكري الأجنبي وأضاف أنه يتعين على المعارضة توحيد صفوفها ومواقفها حتى يتسنى لها المضي قدما في التحول الديمقراطي وشدد إحسان أوغلي على أن منظمة التعاون الإسلامي ترى أن هناك ثلاث قضايا أساسية يجب ألا تحدث أبدا في سوريا ، التدخل العسكري الخارجي ، باعتبار أن لدينا من التجارب ما يكفي لرفض هذا الأمر وأضاف أن القضية الثانية ألا يتحول الصراع الداخلي إلى حرب أهلية أما القضية الثالثة فهي تخص العمل من أجل ألا يتحول الصراع إلى حرب باردة بين توازنات القوى الدولية يو بي آي

--------------------------------------


> بحث ولي العهد السعودي ووزير الدفاع الأمير سلمان بن عبدالعزيز مع وزير الدفاع البريطاني فيليب هاموند الذي يزور المملكة حاليا التعاون العسكري بين البلدين وبحث الجانبان خلال اللقاء آفاق التعاون بين البلدين الصديقين وسبل دعمها وتطويرها ، إضافة إلى استعراض آخر المستجدات على الساحتين الإقليمية والدولية وأكدت وكالة واس أن الأمير سلمان رحب في بداية الاستقبال بالوزير الضيف ، متمنيا له ومرافقيه طيب الإقامة في المملكة وعبر وزير الدفاع البريطاني من جهته عن سعادته بزيارة المملكة ولقائه بنائب خادم الحرمين الشريفين الأمير سلمان بن عبدالعزيز د ب أ

--------------------------------------


> نفت مصادر أمنية مطلعة أمس ، أن تكون السلطات اليمنية وافقت على استحداث قسم أمني في مقر السفارة الأمريكية بصنعاء ، لتنسيق عمليات عسكرية أمريكية مباشرة مع القوات اليمنية المكلفة مكافحة تنظيم القاعدة وأشارت المصادر إلى أن المكتب الأمني الذي تم استحداثه داخل مقر السفارة سيضطلع بمهام تنضوي في إطار التعاون الأمني في مجال مكافحة المخدرات ، وليس من اختصاصه ممارسة أي أنشطة تتعلق بالتعاون القائم بين البلدين في مكافحة تنظيم القاعدة وأكد السفير الأمريكي الجديد بصنعاء جيرالد فيرستاين في أول مؤتمر صحافي يعقده في البلاد أن المكتب الأمني الملحق حديثا بمقر السفارة الأمريكية بصنعاء سيسهم في دعم قدرات الأجهزة الأمنية اليمنية ، من دون الكشف عن طبيعة الدعم الذي سيضطلع بتقديمه وأعلن فيرستاين التزام بلاده بدعم الحكومة اليمنية وشعبها للتغلب على تنظيم القاعدة في شبه الجزيرة العربية وتأمين الحدود اليمنية ومواصلة تدريب قوات مكافحة الإرهاب اليمنية ، وتوفير المعدات اللازمة لها ، للتخلص من التهديدات المباشرة التي تشكلها القاعدة ، وعارض أي حوار مع قادة القاعدة الكبار ، لكنه قال إن حكومته لا تمانع في إجراء حوار مع الأعضاء الجدد بما يمكن من مساعدتهم على التخلص من أفكار التطرف والعنف ، وأكد أن بلاده ستقوم بكل ما تراه مناسبا لمواجهة القاعدة في اليمن وأعلن السفير الأمريكي دعم حكومته لحوار وطني بين السلطة والمعارضة ، وإجراء انتخابات نيابية حرة وشفافة في وقتها المناسب ، وقال نشجع المعارضة والسلطة على الالتزام بالحوار إلى أن يصل إلى نتيجة ناجحة على صعيد آخر ، ألغى عدد من البعثات الدولية التابعة لمنظمات غير حكومية دولية مانحة ، زيارات عمل رسمية لليمن عقب محاولة تنظيم القاعدة استهداف من موظفي السفارة البريطانية ، ومصرع خبير فرنسي يعمل في إحدى شركات التنقيب عن الغاز وعلقت شركة الطيران الألمانية لوفتهانزا رحلاتها إلى المطارات اليمنية حتى إشعار آخر عقب حوادث استهداف الأجانب الأخيرة في اليمن لأسباب وصفتها الشركة بالأمنية المؤقتة في غضون ذلك ، شهدت محاكمة صالح عبدالحبيب الشاووش أحد أخطر عناصر تنظيم القاعدة مشادة كلامية بينه وبين قاضي المحكمة ، إثر طلب الأول النطق بالحكم سريعا وعدم تأجيله ورفضه السماح لمحاميه بتقديم المرافعة الختامية وطالب الشاووش الذي تصنفه السلطات الأمنية اليمنية بأنه من أخطر عناصر القاعدة في اليمن ، قاضي المحكمة الجزائية الابتدائية المتخصصة في قضايا أمن الدولة محسن علوان ، بسرعة النطق في الحكم وعدم إرجائه إلى جلسة أخرى ، فرد القاضي بطلبه من المتهم التزام الهدوء ، مبررا إرجاء النطق بالحكم بوجود إجراءات قانونية يجب على المحكمة التقيد بها ، ما أثار حفيظة المتهم الذي صرخ في القاضي بنبرة حادة مكررا طلبه بالنطق بالحكم قبيل أن يوجه خطابه إلى القاعة ، مستعرضا العمليات الإرهابية التي نفذها واعترف المتهم بصوت مرتفع باشتراكه في تنفيذ سبع هجمات إرهابية استهدفت في معظمها مواقع ومنشآت أمنية وعسكرية في محافظتي حضرموت ومأرب ، واستهداف مصافي النفط بصافر ، وتفجير أنبوب نفط في وادي ساه ، والهجوم على نقاط أمنية ومحاولة اغتيال مدير الأمن السياسي بحضرموت عبدالله الجريزع ، وتجهيز سيارات مفخخة وعبوات ناسفة وجددت النيابة العامة اتهاماتها للمتهم الشاوش بارتكاب جرائم إرهابية باستخدام دراجات نارية واستهداف شخصيات أمنية وعسكرية ومصالح أجنبية في عدد من المدن الرئيسة ، مطالبة بتوقيع أقصى عقوبة يحددها القانون الجزائي واستعرض ممثل النيابة العامة بعض أدلة الإثبات التي اشتملت على ذاكرة محمولة تتضمن مجلدا تحوي آلاف ملف عن صناعة المتفجرات والأسلحة والأحزمة والعبوات الناسفة وندوات عن الجهاد ، وحددت المحكمة الاثنين المقبل موعدا للنطق بالحكم وعقدت المحكمة الجزائية المتخصصة في شؤون الإرهاب في حضرموت شرق أولى جلسات محاكمة يمنيا متهمين بالانتماء إلى القاعدة والتخطيط لاستهداف منشآت عامة وخاصة ونفى المتهمون التهم المنسوبة إليهم ، غير أن بعضهم أقر بتنقلات وسفر بين العراق وأفغانستان وقرر القاضي تأجيل المحاكمة أسبوعين ووقعت اشتباكات بين قوات الأمن بمحافظة شبوة جنوب شرق ومسلحين ، أسفرت عن مصرع مدني وإصابة جنديين ، في منطقة بير علي قرب ميناء بلحاف لتصدير الغاز المسال بمديرية رضوم الساحلية وأكدت مصادر مطلعة أن مدنيا يدعى ناصر باخرخور قتل في الاشتباكات ، وأصيب جنديان تم إسعافهما في مدينة المكلا بمحافظة حضرموت لتلقي العلاج

--------------------------------------


> تمكن الأمن المغربي ، أمس الثلاثاء ، من توقيف عنصرين مواليين لتنظيم داعش الإرهابي وصف أحدهما ب الخطر وينشط في مدينة بركان ، شرقي المملكة ، في إطار التحريات الاستباقية لمواجهة التهديدات الإرهابية ، بينما اتهم مستشارون برلمانيون الحكومة المغربية ب الفشل وعدم توفر الإرادة السياسية لديها للقيام بالإصلاحات وأفاد بلاغ لوزارة الداخلية ، أمس ، بأن التحريات كشفت أن أحد العنصرين بايع الأمير المزعوم لهذا التنظيم الإرهابي ، كان في طور الإعداد لتنفيذ عمليات إرهابية نوعية في المملكة ، بتنسيق مع قادة ميدانيين للتنظم في الساحة السورية والعراقية أما الموقوف الثاني فهو شاب في التاسعة عشرة كان يدرس في مراكش ، وتم توقيفه للاشتباه في تورطه في توزيع منشورات تتضمن دعاية للتنظيم ، وتدوين كتابات حائطية تشيد وتمجد العمليات الإرهابية التي يرتكبها هذا التنظيم وكالات

--------------------------------------


> نفى حزب النور ، الذراع السياسية للدعوة السلفية بمصر ، وجود أي علاقة تربطه بنظيره الفلسطيني ، سواء كانت تنظيمية أو منهجية وقال المتحدث الرسمي باسم الحزب يسري حماد ، أمس ، إن حزبه ليست له أية علاقة بتنظيمات عربية أو دولية فهو كيان مستقل لا يتبع أي تشكيلات خارج مصر ، وأنه لم يسع إلى إحداث أي تحالفات ، حتى وإن حقق نجاحات كبيرة في الداخل

--------------------------------------
